In [1]:
import graphlab

In [2]:
song_data = graphlab.SFrame("song_data.gl/")

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\mlz000\AppData\Local\Temp\graphlab_server_1519366825.log.0


This non-commercial license of GraphLab Create for academic use is assigned to lingzhm@uci.edu and will expire on January 23, 2019.


# Which of the artists below have had the most unique users listening to their songs?

In [3]:
song_data.head()

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


In [7]:
print len(song_data[song_data['artist'] == 'Kanye West']['user_id'].unique())

2522


In [8]:
print len(song_data[song_data['artist'] == 'Foo Fighters']['user_id'].unique())

2055


In [9]:
print len(song_data[song_data['artist'] == 'Taylor Swift']['user_id'].unique())

3246


In [11]:
print len(song_data[song_data['artist'] == 'Lady GaGa']['user_id'].unique())

2928


# Which of the artists below is the most popular artist, the one with highest total listen_count, in the data set?

In [12]:
data = song_data.groupby(key_columns='artist', operations={'total_count': graphlab.aggregate.SUM('listen_count')}).sort('total_count', ascending=False)

In [13]:
data[0]

{'artist': 'Kings Of Leon', 'total_count': 43218L}

# Which of the artists below is the least popular artist, the one with smallest total listen_count, in the data set?

In [14]:
data[-1]

{'artist': 'William Tabbert', 'total_count': 14L}

# Using the first 10,000 unique users only in the test data, use the personalized_model learned on the training data to recommend 1 song to each user. What’s the most recommended song?

In [15]:
train_data,test_data = song_data.random_split(.8,seed=0)

In [16]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [18]:
personalized_model = graphlab.item_similarity_recommender.create(train_data,
                                                         user_id='user_id',
                                                         item_id='song')
recommendations = personalized_model.recommend(subset_test_users,k=1)

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

recommendations finished on 6000/10000 queries. users per second: 73872.5

recommendations finished on 7000/10000 queries. users per second: 75879.9

recommendations finished on 8000/10000 queries. users per second: 77083.9

recommendations finished on 9000/10000 queries. users per second: 75742.3

recommendations finished on 10000/10000 queries. users per second: 69015

In [19]:
groupedData = recommendations.groupby(key_columns='song', operations={'count': graphlab.aggregate.COUNT()})

In [21]:
groupedData.sort('count', ascending=False)

song,count
Secrets - OneRepublic,404
Undo - Björk,388
Revelry - Kings Of Leon,194
You're The One - DwightYoakam ...,173
Fireflies - CharttraxxKaraoke ...,124
Sehr kosmisch - Harmonia,115
Hey_ Soul Sister - Train,95
Horn Concerto No. 4 in Eflat K495: II. Romance ...,88
OMG - Usher featuringwill.i.am ...,62
Dog Days Are Over (RadioEdit) - Florence + The ...,47
